In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [3]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, cc.channel_type_id, om.order_type, pa.type type_p
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_order_risk tor on tor.order_id=om.id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(41364, 67)

## 1_12. 剔除商家数据

In [4]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(38905, 67)

## 1_13.历史订单渠道名称补充

In [5]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

np.int64(0)

## 1_14.渠道归属

In [6]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["channel_name"],x["activity_source"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [7]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [8]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    37791
二手     1114
Name: count, dtype: int64

In [9]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账.xlsx"
path = 'F:/myfile/p站数据/台账数据/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(34635, 2)

是否带锁
未带锁    32030
带锁      2605
Name: count, dtype: int64

(38905, 74)

是否带锁,带锁,未带锁
是否二手,,
二手,422,692
全新,1842,35949


## 1_17.日期处理

In [10]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [11]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
4季度    11213
3季度     9679
1季度     9478
2季度     8535
Name: count, dtype: int64

## 1_21.获取_买断数据

In [12]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(4247, 7)

## 1_22.租后数据 关联 买断数据

In [13]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(38905, 82)

(38905, 78)

In [14]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(5327)

In [15]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      34822
已买断      3992
部分买断       91
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [16]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(2388, 7)

## 1_32.租后数据关联分期取消数据

In [17]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(38905, 86)

(38905, 83)

## 1_41.按风控策略调整时间节点划分时间段

In [18]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [19]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(514198, 23)

In [20]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(514198, 26)

(514198, 23)

In [21]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [22]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [23]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [24]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [25]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(514299)

(514299, 108)

(514198, 23)

(38905, 86)

## 1_62.去重

In [26]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
146250    869632   1.0
146262    869632   1.0
428918    869632   1.0
428930    869632   1.0
146251    869632   2.0
...          ...   ...
478486   2040621  23.0
425710   2040621  24.0
425734   2040621  24.0
478463   2040621  24.0
478487   2040621  24.0

[576 rows x 2 columns]


In [27]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(513867, 108)

In [28]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2.计算逾期天数

## 2_11.异常查询

In [29]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 2_12.实还日期修正

In [30]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [31]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
340722,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
340723,A2024110708472957,1692436,457.98,457.98,2025-03-10,2025-03-10,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT
340724,A2024110708472957,1692436,457.98,457.98,2025-04-10,2025-04-06,3.0,6.0,还款中,NaT,4,NaT,NaN,NaT
472140,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
472141,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT


In [32]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
377803,A20241210113506313,1839860,695.21,665.65,2025-01-13,NaT,1.0,2.0,已逾期,NaT,4,NaT,NaN,NaT
377804,A20241210113506313,1839860,695.21,665.65,2025-02-13,NaT,1.0,3.0,已逾期,NaT,4,NaT,NaN,NaT
377805,A20241210113506313,1839860,695.21,665.65,2025-03-13,NaT,1.0,4.0,已逾期,NaT,4,NaT,NaN,NaT
472140,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
472141,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT


In [33]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [34]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [35]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
202163,A2024050819102519,1043314,1056.37,0.0,2025-10-12,NaT,1.0,18.0,已买断,2025-03-14 21:57:00,8,2025-03-14,NaN,2025-03-14
217157,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14
225261,A20240620152351112,1150772,770.50,0.0,2026-02-24,NaT,1.0,21.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
225262,A20240620152351112,1150772,514.32,0.0,2026-03-24,NaT,1.0,22.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
499259,A20250411105344370,2386604,1082.94,0.0,2026-02-14,NaT,1.0,11.0,已买断,2025-04-19 12:35:48,8,2025-04-19,10.0,2025-04-19


In [36]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [37]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [38]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [39]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [40]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [41]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [42]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [43]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 2_13.获取逾期截止时间

In [44]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-04-21
1   2025-04-21
2   2025-04-21
3   2025-04-21
4   2025-04-21
Name: now_date, dtype: datetime64[s]

## 2_14.重新定义状态

In [45]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    86047
已逾期                    52302
未到首期还款日                32280
续租中                    14600
检测中                       44
Name: count, dtype: int64

订单状态 
租赁中      175078
待归还        6134
已退款        6024
待收货        3636
待发货         255
已完成         220
订单取消         84
待商家收货        42
检测中          41
Name: count, dtype: int64

In [46]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     251098
未到还款日    184397
已逾期支付     36775
已逾期       35353
已取消        6244
Name: count, dtype: int64

In [47]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 2_15.重新定义逾期天数

In [48]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(191517)

Series([], Name: order_number, dtype: object)

In [49]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(513867, 112)

(513867, 112)

In [50]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        52031
还款中         7210
已买断         6321
续租中         5403
已完成          974
检测中          144
未到首期还款日       32
Name: count, dtype: int64

In [51]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 2_16.逾期天数处理

In [52]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 2_17.稽查

In [53]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 2_18.确认是否到表现期

In [54]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [55]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [56]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [57]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [58]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-04-21,1
1,2022-09-04,2022-08-05,2025-04-21,1
2,2022-10-05,2022-09-05,2025-04-21,1
3,2022-11-04,2022-10-05,2025-04-21,1
4,2022-12-05,2022-11-05,2025-04-21,1


In [59]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 2_19.观察日定义 每月月底时间

In [60]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [61]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 2_20.更新逾期天数

In [62]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(901, 2)

label
0.0    503139
1.0     10728
Name: count, dtype: int64

(880, 164)

# 3.逾期加工

In [63]:
# （1）历史最大逾期
df_maxoverdue = df_xz_concat.groupby(['order_number','order_id','下单日期','search_time','归属渠道','是否二手','是否带锁', 'type_p', 'id_card_num', 'mobile']).agg({'逾期天数':'max'}).reset_index()
df_maxoverdue.shape
df_maxoverdue[:1]

(37092, 11)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,type_p,id_card_num,mobile,逾期天数
0,A2023021008550729,125347,2023-02-10,2023-02,CPS直播,全新,未带锁,1.0,532801200203180526,18849606353,15


In [64]:
# （2）FPD SPD TPD
df_xz_concat['当前期数'] = df_xz_concat['当前期数'].astype(int)

df_fstq = df_xz_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape

# df_fstq = df_fstq.rename(columns = {2: 'FPD'})
df_fstq['FPD'] = df_fstq[2]
df_fstq.loc[:,"SPD"] = np.where( df_fstq['FPD'] == 0, df_fstq[3], 0)
df_fstq.loc[:,"TPD"] = np.where((df_fstq['FPD'] == 0) & (df_fstq[3] == 0), df_fstq[4], 0)

df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])][['order_id','FPD','SPD','TPD']]


(38876, 37)

当前期数,order_id,FPD,SPD,TPD
28,12449,0.0,11.0,0.0
3719,508404,0.0,0.0,0.0
24300,1678492,136.0,0.0,0.0
24564,1692916,8.0,0.0,0.0


In [65]:
# （3）第2-4期逾期天数
# df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
# df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
# tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
# df_fstq = df_fstq.fillna(0)
# df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# # df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# # A2023073114301076 508404
# # df_fstq.FPD.max()



df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数', 5:'第五期逾期天数',  6:'第六期逾期天数', 7:'第七期逾期天数'})
df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数','第五期逾期天数','第六期逾期天数','第七期逾期天数']]
tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
df_fstq = df_fstq.fillna(0)

In [66]:
# （4）表关联 
df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type', 'tips']], on = 'order_number', how = 'left')

df_analy_data.shape
df_analy_data = df_analy_data.sort_values(by = ['order_number','overdue_type'])
df_analy_data = df_analy_data.drop_duplicates(subset = 'order_number')
# 以下6个订单号重复,overdue_type 不一样，保留未逾期的，因为 历史最大逾期天数 为0，P站显示为0
# A2024020821520518    2
# A2024010116302721    2
# A2024012715442754    2
# A202401030606401     2
# A2024010216293470    2
# A202402110902322     2
df_analy_data.shape

df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

(37103, 23)

(37092, 23)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,type_p,id_card_num,mobile,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,第五期逾期天数,第六期逾期天数,第七期逾期天数,首期应还日期,overdue_type,tips
2955,A2023073114301076,508404,2023-07-31,2023-07,芝麻租物,全新,未带锁,1.0,371482199409260337,15136842042,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2023-08-03,,"[命中特殊名单验证-一般风险, 免押客户]"
22771,A20241103155713186,1678492,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,6.0,152201200201300017,16223593777,136,136.0,0.0,0.0,136.0,105.0,74.0,46.0,15.0,0.0,2024-11-06,M3,"[3日内GPS(1000)米内下单人数:1, 3日内GPS(200)米内下单人数:1, 3日..."
23032,A20241107103123201,1692916,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,6.0,360427199610221519,15870818200,42,8.0,0.0,0.0,8.0,0.0,40.0,42.0,11.0,0.0,2024-11-10,M1,[免押客户]


In [67]:
# （5）打标签
# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob3pd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
#                                                else 999, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
#                                                else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
#                                                else 999, axis = 1)
# df_analy_data.shape
# var_1 = ['order_number','下单日期','历史最大逾期天数','FPD','SPD','TPD','第二期逾期天数','第三期逾期天数','第四期逾期天数','overdue_type','f2pd7','f3pd7','y_fraud_fpd7','y_fraud_f2pd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])][var_1]



df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# 新增
df_analy_data['mob4dpd'] = np.where(df_analy_data.f3pd7 >= df_analy_data.第五期逾期天数, df_analy_data.f3pd7, df_analy_data.第五期逾期天数)
df_analy_data['mob5dpd'] = np.where(df_analy_data.mob4dpd >= df_analy_data.第六期逾期天数, df_analy_data.mob4dpd, df_analy_data.第六期逾期天数)
df_analy_data['mob6dpd'] = np.where(df_analy_data.mob5dpd >= df_analy_data.第七期逾期天数, df_analy_data.mob5dpd, df_analy_data.第七期逾期天数)

df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
                                               else 0 if x.FPD >= 0 and x.FPD <= 7
                                               else 999, axis = 1)
df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
                                               else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
                                               else 999, axis = 1)
df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
                                               else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
                                               else 999, axis = 1)
df_analy_data['y_credit_mob3dpd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
                                               else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
                                               else 999, axis = 1)
df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
                                               else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
                                               else 999, axis = 1)
# 新增
df_analy_data['y_credit_mob3dpd15'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 15
                                               else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 14
                                               else 999, axis = 1)
df_analy_data['y_credit_mob6dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob6dpd >= 31
                                               else 0 if x.mob6dpd >= 0 and x.mob6dpd <= 30
                                               else 999, axis = 1)

In [68]:
# （6）数据检查
# df_analy_data[df_analy_data.历史最大逾期天数 >0].shape
# df_analy_data[df_analy_data.历史最大逾期天数 >30].shape
# df_analy_data[df_analy_data.历史最大逾期天数 >90].shape
# df_analy_data[df_analy_data.历史最大逾期天数 >180].shape
# df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 <= '2023-12-31')].shape
# df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2024-01-01')&(df_analy_data.下单日期 <= '2024-12-31')].shape
# df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2025-01-01')].shape

# 4.三要素分析

In [69]:
# 打上是否巨量的标签
# 渠道用最新的口径
# 统计

In [70]:
df_analy_data.loc[:, '是否拒量'] = np.where(df_analy_data.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)


In [71]:
# sql_br = ''' -- 百融
# select 
# oa.order_number, rb.verify_rs
# from db_credit.risk_bairong rb
# left join db_credit.order_association oa on oa.trace_id=rb.trace_id
# '''
# df_br = query(sql_br)
# df_br.loc[:, '在网时长'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..TelPeriod.data.value'))
# df_br.loc[:, '是否正常'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..rule_c_telstatus..name_rule'))
# df_br.loc[:, '是否一致'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..rule_c_telcheck_s..name_rule'))
# internet_dict = {
#     '1': '[0, 6)',
#     '2': '[6, 12)',
#     '3': '[12, 24)',
#     '4': '[24, +)]',
#     '手机状态正常': '正常',
#     '手机状态异常:停机、销号等': '异常',
#     '手机三要素简版验证一致': '一致',
#     '手机三要素简版验证不一致': '不一致',
#     '手机三要素简版验证查无此号': '-999'
# }

In [72]:
# 对需要分组的数据进行排序
def sort_value(df):
    dict_time = {
    "[0, 6)": 1,
    "[6, 12)": 2,
    "[12, 24)": 3, 
    "[24, +)]": 4,
    "正常": 5,
    '异常': 6,
    '一致': 7,
    '不一致': 8,
    "-999": 9
    }
    df["时长编码"]=df.index.map(dict_time)
    df.sort_values(by=["时长编码"],inplace=True)
    return df[[0,1]]

In [73]:
# 获取渠道、三要素的数据
def get_zm(df, channel, col, ind, name_key=None):
    if name_key is None:
        df_zm = df[(df.归属渠道==channel)&(df.是否拒量==0)]
        df_pivot = sort_value(pd.pivot_table(df_zm, values='order_number', columns=col, index=ind, aggfunc='nunique'))
    else:
        df_zm = df[(~df.归属渠道.isin(channel))&(df.是否拒量==0)]
        df_pivot = sort_value(pd.pivot_table(df_zm, values='order_number', columns=col, index=ind, aggfunc='nunique'))
    return df_pivot


In [74]:
# df_merge = df_analy_data.merge(df_br, on='order_number', how='left') #[['order_number', '在网时长_new', '是否正常_new', '是否一致_new']]
# df_merge.loc[:, '在网时长_new'] = df_merge.在网时长.apply(lambda x: '-999' if x is None or x==False or x is np.nan else internet_dict[str(x[0])])
# df_merge.loc[:, '是否正常_new'] = df_merge.是否正常.apply(lambda x: '-999' if x is None or x==False or x is np.nan else internet_dict[str(x[0])])
# df_merge.loc[:, '是否一致_new'] = df_merge.是否一致.apply(lambda x: '-999' if x is None or x==False or x is np.nan else internet_dict[str(x[0])])

In [75]:

# # 2023.10-2023.12
# df_merge_1012 = df_merge[(df_merge.下单日期>='2023-10-01')&(df_merge.下单日期<='2023-12-31')]
# # 2024.1-2024.6
# df_merge_0106 = df_merge[(df_merge.下单日期>='2024-01-01')&(df_merge.下单日期<='2024-06-30')]
# # 2024.07 
# df_merge_07 = df_merge[df_merge.search_time=='2024-07']
# # 2024.08
# df_merge_08 = df_merge[df_merge.search_time=='2024-08']
# # 2024.09 
# df_merge_09 = df_merge[df_merge.search_time=='2024-09']
# # 2024.10 
# df_merge_10 = df_merge[df_merge.search_time=='2024-10']
# # 2024.11 
# df_merge_11 = df_merge[df_merge.search_time=='2024-11']
# # 2024.12
# df_merge_12 = df_merge[df_merge.search_time=='2024-12']
# # 2025.01
# df_merge_01 = df_merge[df_merge.search_time=='2025-01']
# # 2025.02 
# df_merge_02 = df_merge[df_merge.search_time=='2025-02']


## 2023.10-2023.12

### 芝麻租物

In [76]:
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit', '在网时长_new')

In [77]:
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit', '是否正常_new')

In [78]:
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_1012, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [79]:
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit', '在网时长_new')

In [80]:
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit', '是否正常_new')

In [81]:
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_1012, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [82]:
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [83]:
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [84]:
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_1012, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.1-2024.6

### 芝麻租物

In [85]:
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit', '在网时长_new')

In [86]:
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit', '是否正常_new')

In [87]:
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_0106, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [88]:
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit', '在网时长_new')

In [89]:
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit', '是否正常_new')

In [90]:
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_0106, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [91]:
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [92]:
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [93]:
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_0106, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.07 

### 芝麻租物

In [94]:
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit', '在网时长_new')

In [95]:
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit', '是否正常_new')

In [96]:
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_07, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_07, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [97]:
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit', '在网时长_new')

In [98]:
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit', '是否正常_new')

In [99]:
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_07, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_07, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [100]:
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [101]:
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [102]:
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_07, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.08

### 芝麻租物

In [103]:
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit', '在网时长_new')

In [104]:
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit', '是否正常_new')

In [105]:
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_08, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_08, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [106]:
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit', '在网时长_new')

In [107]:
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit', '是否正常_new')

In [108]:
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_08, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_08, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非巨量

In [109]:
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [110]:
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [111]:
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_08, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.09 

### 芝麻租物

In [112]:
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit', '在网时长_new')

In [113]:
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit', '是否正常_new')

In [114]:
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_09, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_09, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [115]:
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit', '在网时长_new')

In [116]:
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit', '是否正常_new')

In [117]:
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_09, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_09, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [118]:
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [119]:
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [120]:
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_09, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.10

### 芝麻租物

In [121]:
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit', '在网时长_new')

In [122]:
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit', '是否正常_new')

In [123]:
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_10, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_10, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [124]:
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit', '在网时长_new')

In [125]:
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit', '是否正常_new')

In [126]:
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_10, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_10, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [127]:
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [128]:
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [129]:
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_10, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.11

### 芝麻租物

In [130]:
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit', '在网时长_new')

In [131]:
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit', '是否正常_new')

In [132]:
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_11, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_11, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [133]:
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit', '在网时长_new')

In [134]:
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit', '是否正常_new')

In [135]:
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_11, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_11, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [136]:
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [137]:
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [138]:
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_11, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2024.12

### 芝麻租物

In [139]:
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit', '在网时长_new')

In [140]:
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit', '是否正常_new')

In [141]:
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_12, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_12, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [142]:
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit', '在网时长_new')

In [143]:
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit', '是否正常_new')

In [144]:
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_12, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_12, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [145]:
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [146]:
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [147]:
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_12, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2025.01

### 芝麻租物

In [148]:
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit', '在网时长_new')

In [149]:
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit', '是否正常_new')

In [150]:
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_01, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_01, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [151]:
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit', '在网时长_new')

In [152]:
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit', '是否正常_new')

In [153]:
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_01, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_01, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [154]:
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [155]:
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [156]:
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_01, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

## 2025.02 

### 芝麻租物

In [157]:
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit', '在网时长_new')

In [158]:
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit', '是否正常_new')

In [159]:
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_02, '芝麻租物', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_02, '芝麻租物', 'y_credit', '是否一致_new')

### 搜索渠道

In [160]:
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_fpd7', '在网时长_new')
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_f3pd7', '在网时长_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit_mob3pd30', '在网时长_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit', '在网时长_new')

In [161]:
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_fpd7', '是否正常_new')
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_f3pd7', '是否正常_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit_mob3pd30', '是否正常_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit', '是否正常_new')

In [162]:
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_fpd7', '是否一致_new')
# get_zm(df_merge_02, '搜索渠道', 'y_fraud_f3pd7', '是否一致_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit_mob3pd30', '是否一致_new')
# get_zm(df_merge_02, '搜索渠道', 'y_credit', '是否一致_new')

### 非芝麻非搜索非抖音非拒量

In [163]:
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '在网时长_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '在网时长_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '在网时长_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '在网时长_new', '抖音')

In [164]:
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否正常_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否正常_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否正常_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否正常_new', '抖音')

In [165]:
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_fpd7', '是否一致_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_fraud_f3pd7', '是否一致_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit_mob3pd30', '是否一致_new', '抖音')
# get_zm(df_merge_02, ['芝麻租物', '搜索渠道', '抖音渠道'], 'y_credit', '是否一致_new', '抖音')

# 京享租

身份证+手机号为md5加密，下单日期、是否发货、逾期标签明文哈

## 出库订单

In [166]:
# 身份证+手机号为md5加密，下单日期、是否发货、逾期标签明文哈
df_analy_data_jxz = df_analy_data[(df_analy_data.type_p!=4)&(df_analy_data.是否拒量==0)&(df_analy_data.归属渠道!='抖音渠道')&(df_analy_data.search_time>='2024-07')&(df_analy_data.search_time<='2025-02')]

df_analy_data_jxz.loc[:, '是否发货'] = '已发货'
df_analy_data_jxz = df_analy_data_jxz[['下单日期', 'id_card_num', 'mobile', '是否发货' , 'y_fraud_fpd7', 'y_credit_mob3dpd15', 'y_credit_mob6dpd30']].rename(columns={'y_fraud_fpd7': 'label_1', 'y_credit_mob3dpd15': 'label_2', 'y_credit_mob6dpd30': 'label_3'})


## 免押进件样本

In [167]:
sql_my = '''
select
om.create_time, om.order_number, om.order_type, pa.type,topay.total_freeze_fund_amount,pa.name as activity_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
, tmu.id_card_num, tmu.mobile, om.merchant_name,tod.sku_attributes,top.total_describes, tmu.true_name, om.status
from db_digua_business.t_order om
left join db_digua_business.t_order_risk tor on tor.order_id=om.id
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 免押
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 支付宝用户表
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
-- 订单详情表
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 备注信息合并
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
where date_format(om.create_time, '%Y-%m-%d')>='2025-04-01'
and date_format(om.create_time, '%Y-%m-%d')<='2025-04-15'
'''
df_my = query(sql_my)

In [168]:
df_my["下单日期"]=df_my["create_time"].dt.date
df_my["下单日期"]=pd.to_datetime(df_my["下单日期"],errors="coerce")

In [169]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df_my["状态编码"]=df_my["status2"].map(dict_status_code)
df_my.sort_values(by=["下单日期","状态编码"],inplace=True)
#删除重复单号
df_my.drop_duplicates(subset=["order_number"],inplace=True)
#删除身份证空值行
df_my.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df_my.drop(df_my[df_my['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df_my.drop(df_my[df_my['activity_name']=="1000单秘密计划"].index,inplace=True)
df_my.drop(df_my[df_my['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df_my.drop(df_my[df_my['activity_name']=="1000单曙光计划"].index,inplace=True)
df_my.drop(df_my[df_my['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df_my.dropna(subset=["status2"],axis=0,inplace=True)
#删除重复订单
df_my.drop_duplicates(subset=["true_name","mobile","id_card_num","下单日期"],keep="last",inplace=True)
df_my.drop(df_my[df_my['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

In [170]:
df_my = df_my[df_my.type!=4]
df_my = clean.drop_merchant(df_my)
df_my = clean.drop_rejected_merchant(df_my)
df_my.loc[:, '是否拒量'] = np.where(df_my.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
df_my.loc[:, '抖音渠道'] = np.where((df_my.activity_name.str.contains('抖音'))&(df_my.order_type=='DY_ORDER'), 1, 0)
df_my.loc[:, "押金类型"] = np.where(df_my["total_freeze_fund_amount"]>0,"部分免押","全免押")
df_my.loc[:,"进件"]=np.where((df_my["status2"]=="待支付")|(df_my["status2"]=="订单取消"),"未进件","进件")
df_my.loc[:, '是否发货'] = np.where(df_my.status.isin([2, 3, 4, 5, 6, 8, 15]), '已发货', '未发货')
df_my_new = df_my[(df_my.押金类型=='全免押')&(df_my.进件=="进件")] # &(df_my.是否拒量==0)&(df_my.抖音渠道==0)

In [174]:
df_my_new = df_my_new[['下单日期', 'id_card_num', 'mobile', '是否发货']]
df_my_new.loc[:, 'label_1'] = np.nan
df_my_new.loc[:, 'label_2'] = np.nan
df_my_new.loc[:, 'label_3'] = np.nan

df_jxz = pd.concat([df_analy_data_jxz, df_my_new], axis=0)
df_jxz.loc[:, 'id_card_num_MD5'] = df_jxz.id_card_num.apply(lambda x: clean.md5_hash(x))
df_jxz.loc[:, 'mobile_MD5'] = df_jxz.mobile.apply(lambda x: clean.md5_hash(x))
df_jxz = df_jxz[['id_card_num_MD5', 'mobile_MD5', '下单日期', '是否发货', 'label_1', 'label_2', 'label_3']]
# df_jxz["下单日期"]=pd.to_datetime(df_jxz["下单日期"],errors="coerce")
df_jxz["下单日期"]=df_jxz["下单日期"].astype('str')
with pd.ExcelWriter('F:/需求/总需求/京享租.xlsx', engine='xlsxwriter') as writer:
    df_jxz.to_excel(writer, sheet_name='京享租', index=False)
df_jxz

,id_card_num_MD5,mobile_MD5,下单日期,是否发货,label_1,label_2,label_3
14275,46d2a5b6cacbedd1d04b6cf680e71392,a8b337a287825057b8d30f620e717033,2024-07-01,已发货,0.0,0.0,0.0
14276,8682e5cd3a9b52510192feb6e02b8eaa,52173070c0ea2e7df4a50a8212d24b00,2024-07-01,已发货,0.0,0.0,0.0
14277,b79c6fa90b20cd41ab3fc05e3671553f,85172b14e17aa8b189bab122ad22cb9e,2024-07-01,已发货,0.0,0.0,0.0
14278,3b3ffa0206cc21de911f3a9cdd5cffb0,9aa9737fcff1d4cd0ef72e9d39753324,2024-07-01,已发货,0.0,0.0,0.0
14279,f379de0ae07c779586f563cc654ed399,80b32a5b147b239767c27194b85d3c87,2024-07-01,已发货,0.0,1.0,1.0
...,...,...,...,...,...,...,...
80232,f4ae01bf237040407de2f134fdc866b8,60441a352b9a75f69e37759d5902e777,2025-04-15,已发货,NaN,NaN,NaN
80359,63fc9b7392e80774aa3b581a936ac0c3,d1c5d590935277aa304fe9eea918ec57,2025-04-15,已发货,NaN,NaN,NaN
80363,2ee684dc59cb85521c1f061999240b64,0127d5d759c870271357663211e1602c,2025-04-15,已发货,NaN,NaN,NaN
80408,ae0642b08ae327f668c81d00ad77bdc2,e94557f6738cf97c32c80ddf298d1a3c,2025-04-15,已发货,NaN,NaN,NaN
